In [3]:
import numpy as np
import matplotlib.pyplot as plt 
import Mesh
import math
import MES

In [4]:
def find_real_space_coordinate(global_number,N, a):
    y = math.floor((global_number-1)/ (2*N+1))
    x = (global_number + y) % (2*N+1 + 1) - 1
    return ((x-N)*a, (y-N)*a)

N = 10
L = 100 #[nm]
a = L/(2*N)
m = 0.067
omega = 10. / 27211.6
global_numbers = np.zeros((2*N+1, 2*N+1))
global_numbers = Mesh.create_global_numbers(N,global_numbers)
for i in range(1,4*N**2 + 1):
    for j in range(1,5):
        #print((i,j,Mesh.find_global_number(i,j,N)))
        number = Mesh.find_global_number(i,j,N)
        #print(find_real_space_coordinate(number,N,a))
        

MES.MES(0.1,N,a, m, omega)

-1.6537065075185987e-05 0.024597230592835414
-1.6537065075185987e-05 0.024615605109585624
-1.6537065075185987e-05 0.024633979626335834
-1.6537065075185987e-05 0.02465235414308604
-1.6537065075185987e-05 0.024670728659836254
-1.6537065075185987e-05 0.02468910317658646
-1.6537065075185987e-05 0.02470747769333667
-1.6537065075185987e-05 0.024725852210086878
-1.6537065075185987e-05 0.02474422672683709
-1.6537065075185987e-05 0.0247626012435873
-1.6537065075185987e-05 0.02478097576033751
-1.6537065075185987e-05 0.02479935027708772
-1.6537065075185987e-05 0.024817724793837925
-1.6537065075185987e-05 0.02483609931058814
-1.6537065075185987e-05 0.024854473827338346
-1.6537065075185987e-05 0.024872848344088556
-1.6537065075185987e-05 0.024891222860838766
-1.6537065075185987e-05 0.024909597377588976
-1.6537065075185987e-05 0.024927971894339183
-1.6537065075185987e-05 0.024946346411089396
-1.6537065075185987e-05 0.024964720927839603
1.8374516750234797e-06 0.024597230592835414
1.8374516750234797e-